In [72]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf
!pip install streamlit

In [7]:
import os
import shutil
import pikepdf

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

# Função para verificar e criar a hierarquia de pastas
def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Pasta criada: {folder}")
        else:
            print(f"Pasta já existe: {folder}")

# Cria a hierarquia de pastas
ensure_folders_exist()


# Busca o primeiro arquivo PDF na pasta maps
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]

if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta maps.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")

    try:
        # Tenta descriptografar o PDF
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso! Salvo em: {decrypted_pdf_path}")

        # Move o PDF criptografado para a pasta processed
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
        print(f"PDF original criptografado movido para: {processed_folder}")

    except pikepdf.PasswordError:
        print(f"O PDF '{pdf_files[0]}' está protegido por senha e não pode ser desbloqueado sem ela.")
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


from PyPDF2 import PdfReader
import os
import shutil
import re

# Paths das pastas
input_folder = "./maps/decrypted"
processed_folder = "maps/decrypted/processed"

# Verificar se a pasta de processados existe, caso contrário, criá-la
os.makedirs(processed_folder, exist_ok=True)

# Obter a lista de PDFs na pasta de entrada
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

if pdf_files:
    # Processar cada PDF na pasta
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        
        try:
            # Inicializar o texto completo para o PDF atual
            full_text = ""
            
            # Ler o PDF
            reader = PdfReader(input_pdf_path)
            
            # Iterar sobre as páginas e adicionar os delimitadores
            for idx, page in enumerate(reader.pages):
                full_text += f"------------------ PAGINA {idx} ----------------\n"
                full_text += page.extract_text()
            
            # Dividir o texto baseado nos delimitadores
            paginas = re.split(r"------------------ PAGINA \d+ ----------------", full_text)
            paginas = [pagina.strip() for pagina in paginas if pagina.strip()]
            
            # Exibir a quantidade de páginas úteis detectadas
            print(f"\nTotal de páginas detectadas em '{file_name}': {len(paginas)}\n")
            
            # Criar dicionário para armazenar o conteúdo das páginas
            texto_paginas = {}
            for idx, pagina in enumerate(paginas, start=1):
                texto_paginas[f"texto_pagina{idx}"] = pagina
            
            # Você pode processar o dicionário `texto_paginas` aqui, se necessário
            
        except Exception as e:
            print(f"Ocorreu um erro ao processar '{file_name}': {e}")
        finally:
            # Mover o PDF processado para a pasta de processados
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para a pasta de processados: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

import re
import pandas as pd

# Regexes para capturar os elementos de interesse
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Montantes\s+Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s+\d+\s+([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})\s+Em litígio judicial"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})\s+Tipo")
}

# Função para aplicar regex em texto
def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None
 
# Inicializar lista para armazenar os dados
data = []

# Iterar sobre as páginas do dicionário texto_paginas
for page_number, page_text in texto_paginas.items():
    # Extrair informações de cada página usando regex
    row = {
        'pagina_pdf': page_number,
        'nome': get_feature(page_text, 'nome'),
        'nif': get_feature(page_text, 'nif'),
        'mes_mapa': get_feature(page_text, 'mes_mapa'),
        'instituicao': get_feature(page_text, 'instituicao'),
        'divida': get_feature(page_text, 'total_em_divida'),
        'litigio': get_feature(page_text, 'litigio'),
        'parcela': get_feature(page_text, 'abatido_ativo'),
        'garantias': get_feature(page_text, 'garantias'),
        'num_devedores': get_feature(page_text, 'num_devedores'),
        'prod_financeiro': get_feature(page_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(page_text,'entrada_incumpr'),
        'dat_inicio': get_feature(page_text, 'dat_inicio'),
        'dat_fim': get_feature(page_text, 'dat_fim')
    }
    # Adicionar os dados ao conjunto
    data.append(row)

# Criar o DataFrame
df = pd.DataFrame(data)
df.iloc[:-1]

Pasta já existe: ./maps/encrypted/
Pasta já existe: ./maps/encrypted/processed
Pasta já existe: ./maps/decrypted/
Pasta já existe: ./maps/decrypted/processed
PDF desbloqueado com sucesso! Salvo em: ./maps/decrypted/decrypted_0aacb97a-dc7e-4ee3-ae78-1c3de322f848.pdf
PDF original criptografado movido para: ./maps/encrypted/processed

Total de páginas detectadas em 'decrypted_0aacb97a-dc7e-4ee3-ae78-1c3de322f848.pdf': 7

PDF 'decrypted_0aacb97a-dc7e-4ee3-ae78-1c3de322f848.pdf' movido para a pasta de processados: maps/decrypted/processed/decrypted_0aacb97a-dc7e-4ee3-ae78-1c3de322f848.pdf


,pagina_pdf,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,texto_pagina1,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,"CAIXA GERAL DE DEPÓSITOS, S.A. (0035)","93 714,77",Não,"461,87","130 000,00",3,Crédito à habitação,None,2007-07-16,2043-08-08
1,texto_pagina2,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,"BBVA, INSTITUIÇÃO FINANCEIRA DE CRÉDITO S.A. (...","0,00",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2012-10-29,9999-12-31
2,texto_pagina3,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,ONEY BANK - SUCURSAL EM PORTUGAL (0881),"1 736,70",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2013-10-10,9999-12-31
3,texto_pagina4,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)","3 324,11",Não,"116,25",None,1,Crédito pessoal,None,2019-07-31,2027-07-31
4,texto_pagina5,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","3 262,49",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2019-10-30,9999-12-31
5,texto_pagina6,ANA CRISTINA FERREIRA TOMAS DA SILVA CAMILO,188684093,outubro de 2024,"BANCO PRIMUS, SA (0246)","11 558,33",Não,"152,93","9 550,00",3,Crédito automóvel (excluíndo locações financei...,None,2024-05-07,2034-06-05


In [4]:
import os
import shutil
import pikepdf

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")
f2_senninha_folder = "./maps/f2_senninha/"

# Função para verificar e criar a hierarquia de pastas
def ensure_folders_exist():
    folders = [
        source_folder,
        processed_folder,
        decrypted_folder,
        decrypted_processed_folder,
        f2_senninha_folder,  # Inclui a nova pasta
    ]
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Pasta criada: {folder}")
        else:
            print(f"Pasta já existe: {folder}")

# Cria a hierarquia de pastas
ensure_folders_exist()

# Busca o primeiro arquivo PDF na pasta maps
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]

if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta maps.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")

    try:
        # Tenta descriptografar o PDF
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso! Salvo em: {decrypted_pdf_path}")

        # Move o PDF criptografado para a pasta processed
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
        print(f"PDF original criptografado movido para: {processed_folder}")

    except pikepdf.PasswordError:
        print(f"O PDF '{pdf_files[0]}' está protegido por senha e não pode ser desbloqueado sem ela.")
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


from PyPDF2 import PdfReader
import os
import shutil
import re

# Paths das pastas
input_folder = "./maps/decrypted"
processed_folder = "maps/decrypted/processed"

# Verificar se a pasta de processados existe, caso contrário, criá-la
os.makedirs(processed_folder, exist_ok=True)

# Obter a lista de PDFs na pasta de entrada
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

if pdf_files:
    # Processar cada PDF na pasta
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        
        try:
            # Inicializar o texto completo para o PDF atual
            full_text = ""
            
            # Ler o PDF
            reader = PdfReader(input_pdf_path)
            
            # Iterar sobre as páginas e adicionar os delimitadores
            for idx, page in enumerate(reader.pages):
                full_text += f"------------------ PAGINA {idx} ----------------\n"
                full_text += page.extract_text()
            
            # Dividir o texto baseado nos delimitadores
            paginas = re.split(r"------------------ PAGINA \d+ ----------------", full_text)
            paginas = [pagina.strip() for pagina in paginas if pagina.strip()]
            
            # Exibir a quantidade de páginas úteis detectadas
            print(f"\nTotal de páginas detectadas em '{file_name}': {len(paginas)}\n")
            
            # Criar dicionário para armazenar o conteúdo das páginas
            texto_paginas = {}
            for idx, pagina in enumerate(paginas, start=1):
                texto_paginas[f"texto_pagina{idx}"] = pagina
            
            # Você pode processar o dicionário `texto_paginas` aqui, se necessário
            
        except Exception as e:
            print(f"Ocorreu um erro ao processar '{file_name}': {e}")
        finally:
            # Mover o PDF processado para a pasta de processados
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para a pasta de processados: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

import re
import pandas as pd

# Regexes para capturar os elementos de interesse
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Montantes\s+Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s+\d+\s+([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})\s+Em litígio judicial"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})\s+Tipo")
}

# Função para aplicar regex em texto
def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None
 
# Inicializar lista para armazenar os dados
data = []

# Iterar sobre as páginas do dicionário texto_paginas
for page_number, page_text in texto_paginas.items():
    # Extrair informações de cada página usando regex
    row = {
        'pagina_pdf': page_number,
        'nome': get_feature(page_text, 'nome'),
        'nif': get_feature(page_text, 'nif'),
        'mes_mapa': get_feature(page_text, 'mes_mapa'),
        'instituicao': get_feature(page_text, 'instituicao'),
        'divida': get_feature(page_text, 'total_em_divida'),
        'litigio': get_feature(page_text, 'litigio'),
        'parcela': get_feature(page_text, 'abatido_ativo'),
        'garantias': get_feature(page_text, 'garantias'),
        'num_devedores': get_feature(page_text, 'num_devedores'),
        'prod_financeiro': get_feature(page_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(page_text,'entrada_incumpr'),
        'dat_inicio': get_feature(page_text, 'dat_inicio'),
        'dat_fim': get_feature(page_text, 'dat_fim')
    }
    # Adicionar os dados ao conjunto
    data.append(row)

# Criar o DataFrame
df = pd.DataFrame(data)
df.iloc[:-1]

df

#### TRATAMENTO DE DADOS DO SENNINHA

# Tratamento da coluna divida
df['divida'] = df['divida'].astype(str)  # Converte para string
df['divida'] = df['divida'].str.replace(' ', '').str.replace(',', '.')  # Remove caracteres indesejados e ajusta separadores
df['divida'] = pd.to_numeric(df['divida'], errors='coerce')  # Converte para numérico, substituindo erros por NaN
df['divida'] = df['divida'].fillna(0.0)  # Substitui NaN por 0.0

# Tratamento da coluna parcela
df['parcela'] = df['parcela'].astype(str)  # Converte para string
df['parcela'] = df['parcela'].str.replace(' ', '').str.replace(',', '.')  # Remove caracteres indesejados e ajusta separadores
df['parcela'] = pd.to_numeric(df['parcela'], errors='coerce')  # Converte para numérico, substituindo erros por NaN
df['parcela'] = df['parcela'].fillna(0.0)  # Substitui NaN por 0.0

# Tratamento da coluna garantias
df['garantias'] = df['garantias'].astype(str)  # Converte para string
df['garantias'] = df['garantias'].str.replace(' ', '').str.replace(',', '.')  # Remove caracteres indesejados e ajusta separadores
df['garantias'] = pd.to_numeric(df['garantias'], errors='coerce')  # Converte para numérico, substituindo erros por NaN
df['garantias'] = df['garantias'].fillna(0.0)  # Substitui NaN por 0.0

df

#PERFILADOR SENNINHA O TERROR DOS GRINGOS

# Aplicando o segundo filtro onde a coluna "litigio" é igual a "Não"
df_litigio= df[df["litigio"] == "Não"]

# Lista de opções para filtro
opcoes_prod_financeiro = [
    "Cartão de crédito - sem período de free-float",
    "Cartão de crédito - com período de free-float",
    "Crédito automóvel (excluíndo locações financeiras)",
    "Crédito pessoal",
    "Crédito renovável - Linha de crédito",
    "Cartão de crédito",
    "Outros créditos",
    "Ultrapassagens de crédito",
    "Crédito renovável - conta corrente bancária",
    "Facilidades de descoberto",
    "Cartão de crédito - cartão de débito diferido"
]

# Aplicando o primeiro filtro onde a coluna "produtos financeiros" é igual aos prod_financeiros que perfilam
df_prod_fin = df_litigio[df_litigio["prod_financeiro"].isin(opcoes_prod_financeiro)]

# Aplicando o segundo filtro onde a coluna "litigio" é igual a "Não"
df_prod_fin = df_prod_fin[df_prod_fin["litigio"] == "Não"]

# Aplicado o terceiro filtro onde a coluna "garantias" é menor ou igual a 0.0
df_prod_fin = df_prod_fin[df_prod_fin["garantias"] <= 0]


df_perfilados_agg = (
    df_prod_fin.groupby("nif", as_index=False)
    .agg({
        "nome": "first",
        "mes_mapa": "first",
        "litigio": "first",
        "divida": "sum",
        "garantias": "sum"
    })
)

df_perfilados_agg

def controle_fluxo(df_perfilados_agg, processed_pdf_path):
    """
    Verifica se o valor da coluna 'divida' em df_perfilados_agg (registro único) é maior ou igual a 6000.
    Se for, copia o arquivo especificado em processed_pdf_path para o diretório de destino.

    Args:
        df_perfilados_agg (DataFrame): DataFrame contendo a coluna 'divida' com apenas um registro.
        processed_pdf_path (str): Caminho completo do arquivo a ser copiado.
    """
    # Diretório de destino
    destination_dir = "maps/f2_senninha"

    # Verifica se o destino existe, caso contrário, cria-o
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Verifica se o valor de 'divida' atende à condição
    if df_perfilados_agg.loc[0, "divida"] >= 6000:
        # Extrai o nome do arquivo do caminho fornecido
        file_name = os.path.basename(processed_pdf_path)
        destination_path = os.path.join(destination_dir, file_name)

        # Verifica se o arquivo de origem existe e realiza a cópia
        if os.path.exists(processed_pdf_path):
            shutil.copy(processed_pdf_path, destination_path)
            print(f"Arquivo {file_name} copiado para {destination_dir}.")
        else:
            print(f"Arquivo {processed_pdf_path} não encontrado.")
    else:
        print("A dívida é menor que 6000. Nenhuma ação realizada.")


controle_fluxo(df_perfilados_agg, processed_pdf_path)

df_perfilados_agg

Pasta já existe: ./maps/encrypted/
Pasta já existe: ./maps/encrypted/processed
Pasta já existe: ./maps/decrypted/
Pasta já existe: ./maps/decrypted/processed
Pasta já existe: ./maps/f2_senninha/
PDF desbloqueado com sucesso! Salvo em: ./maps/decrypted/decrypted_0b8c1305-52fe-4f7f-b5c9-7d6d3c9a1ccc.pdf
PDF original criptografado movido para: ./maps/encrypted/processed

Total de páginas detectadas em 'decrypted_0b8c1305-52fe-4f7f-b5c9-7d6d3c9a1ccc.pdf': 0

PDF 'decrypted_0b8c1305-52fe-4f7f-b5c9-7d6d3c9a1ccc.pdf' movido para a pasta de processados: maps/decrypted/processed/decrypted_0b8c1305-52fe-4f7f-b5c9-7d6d3c9a1ccc.pdf


KeyError: 'divida'

In [110]:
df


,pagina_pdf,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,texto_pagina1,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"BANCO INVEST, SA (0014)",207.22,Não,0.00,210.0,1,Outros créditos,None,2019-11-18,2020-10-18
1,texto_pagina2,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,None,123.32,Não,1.44,120.0,1,Outros créditos,None,2022-12-22,2024-12-22
2,texto_pagina3,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"CAIXA ECONÓMICA MONTEPIO GERAL, CAIXA ECONÓMIC...",924.29,Não,0.00,0.0,1,Cartão de crédito - com período de free-float,None,2023-01-10,9999-12-31
3,texto_pagina4,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"BANKINTER CONSUMER FINANCE, E.F.C., SA - SUCUR...",4412.26,Não,0.00,0.0,1,Cartão de crédito - com período de free-float,2024-09-20,2023-02-23,9999-12-31
4,texto_pagina5,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,COFIDIS (0921),4000.34,Não,175.76,0.0,1,Crédito pessoal,2024-08-26,2023-07-27,2026-10-25
5,texto_pagina6,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"BNP PARIBAS PERSONAL FINANCE, S.A. - SUCURSAL ...",3099.21,Não,76.13,0.0,2,Crédito automóvel (excluíndo locações financei...,None,2023-08-10,2028-08-05
6,texto_pagina7,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"UNICRE - INSTITUIÇÃO FINANCEIRA DE CRÉDITO, S....",432.68,Não,85.87,0.0,1,Cartão de crédito - com período de free-float,None,2023-08-07,9999-12-31
7,texto_pagina8,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,YOUNITED S.A. - SUCURSAL EM PORTUGAL (3574),7108.49,Não,131.93,0.0,1,Crédito pessoal,2024-09-01,2024-01-12,2030-02-01
8,texto_pagina9,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)",3588.91,Não,49.85,0.0,1,Crédito pessoal,None,2024-09-27,2031-09-27
9,texto_pagina10,MARIA PAULA PEQUITO DE ALMEIDA SAMPAIO SOARES ...,175011982,setembro de 2024,None,0.00,None,0.00,0.0,None,None,None,None,None


In [114]:
print(page_text)

Tipo de Responsabilidade: 
Devedor
Resumo das Responsabilidades de Crédito relativas a 31 de outubro de 2024
Conjunto
Montante em dívida
Total
Em incumprimento
Montante Potencial
N.º Produtos
Produtos c/ garantia
Facilidades de 
descoberto
750,99 €
0,00 €
749,01 €
1
0
750,99 €
0,00 €
749,01 €
1
0
Individual
Montante em dívida
Total
Em incumprimento
Montante Potencial
N.º Produtos
Produtos c/ garantia
Cartão de crédito - com 
período de free-float
854,15 €
0,00 €
0,00 €
1
0
Cartão de crédito - sem 
período de free-float
2 498,02 €
0,00 €
1,98 €
1
0
Crédito pessoal 
28 568,81 €
0,00 €
0,00 €
2
0
31 920,98 €
0,00 €
1,98 €
4
0
Instituições e produtos financeiros
Nº de instituições que comunicaram informação:
3
Nº de instituições que comunicaram incumprimento:
0
Nº total de produtos financeiros comunicados:
5
Fim de relatório
A informação  prestada,  extraída  da  Central  de  Responsabilidades  de  Crédito (CRC)  no  momento  em  que  a  consulta  foi efetuada,  é  da 
responsabilidade das

DEVELOP PERFILADOR

In [82]:
#PERFILADOR SENNINHA 

# Aplicando o segundo filtro onde a coluna "litigio" é igual a "Não"
df_litigio= df[df["litigio"] == "Não"]

# Lista de opções para filtro
opcoes_prod_financeiro = [
    "Cartão de crédito - sem período de free-float",
    "Cartão de crédito - com período de free-float",
    "Crédito automóvel (excluíndo locações financeiras)",
    "Crédito pessoal",
    "Crédito renovável - Linha de crédito",
    "Cartão de crédito",
    "Outros créditos",
    "Ultrapassagens de crédito",
    "Crédito renovável - conta corrente bancária",
    "Facilidades de descoberto",
    "Cartão de crédito - cartão de débito diferido"
]

# Aplicando o primeiro filtro onde a coluna "produtos financeiros" é igual aos prod_financeiros que perfilam
df_prod_fin = df_litigio[df_litigio["prod_financeiro"].isin(opcoes_prod_financeiro)]

# Aplicando o segundo filtro onde a coluna "litigio" é igual a "Não"
df_prod_fin = df_prod_fin[df_prod_fin["litigio"] == "Não"]

# Aplicado o terceiro filtro onde a coluna "garantias" é menor ou igual a 0.0
df_prod_fin = df_prod_fin[df_prod_fin["garantias"] <= 0]

# somando dividas perfiladas
df_perfilados_agg = (
    df_prod_fin.groupby("nif", as_index=False)
    .agg({
        "nome": "first",
        "mes_mapa": "first",
        "litigio": "first",
        "divida": "sum",
        "garantias": "sum"
    })
)

df_perfilados_agg, file_name

(         nif                               nome         mes_mapa litigio  \
 0  271878061  FABIO JOSE ROSA DOS SANTOS CATITA  janeiro de 2024     Não   
 
     divida  garantias  
 0  4127.73        0.0  ,
 'decrypted_09f6484e-1c43-472c-8f8d-9e4d69581026.pdf')